In [ ]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
from skimage import io, filters
import numpy as np

from tqdm.notebook import tqdm

from PIL import Image
import numpy as np

import cv2
import copy
import time

from mpire import WorkerPool

from PIL import Image
from matplotlib import cm
from tqdm.notebook import tqdm


from combra import stats as cstats
from combra import approx as capprox
from combra import image as cimage

from numba import njit
import numpy as np
import cv2
import os

from combra.tests import test_fractal_dimensions
from combra.contours import contour_to_binary_mask, scale_contour, draw_contours

from pathlib import Path
import re

import pyarrow as pa
import pyarrow.parquet as pq




In [ ]:
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (3-10-2024_18-11--6-46)/IMG_1259.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (3-10-2024_18-11--6-46)/IMG_2266.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (24-09-2024_12-27--1-0)/IMG_6186.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (24-09-2024_12-27--1-0)/IMG_5589.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (12-09-2024_12-26--1-3)/IMG_4097.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (12-09-2024_12-26--1-3)/IMG_3603.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (1-10-2024_14-6--2-40)/IMG_0585.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (1-10-2024_14-6--2-40)/IMG_0268.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (09-09-2024_12-10--0-44)/IMG_9009.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (09-09-2024_12-10--0-44)/IMG_8955.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (02-09-2024_13-33--2-7)/IMG_8747.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (02-09-2024_13-33--2-7)/IMG_6932.JPG'

# img = io.imread(image_path)
# img = img[:,900:-900]

# original_img = original_img[500:-300,1200:-1500]
# original_img=original_img[:2000,:2000]

# plt.imshow(img)
# plt.show()

# Image preprocess

In [ ]:
image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (09-09-2024_12-10--0-44)/IMG_8955.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (02-09-2024_13-33--2-7)/IMG_8747.JPG'


img = io.imread(image_path)
img = img[500:-300,1200:-1500]

img=img[:2000,:2000]
img[:250,:250]=np.full((250,250,3),20)

plt.imshow(img)
plt.show()

In [ ]:
images_paths=['images/0001.JPG',
              'images/0500.JPG',
              'images/1000.JPG',
              'images/1500.JPG',
              'images/2000.JPG',
              'images/2500.JPG',
              'images/3000.JPG',
              'images/3500.JPG',
              'images/4000.JPG',
            ]
img = io.imread(images_paths[2])
img = img[400:-400,1300:-1300]

print(img.shape)


thresh = filters.threshold_otsu(img)
binary = img > thresh  # Apply threshold
binary = binary*255


plt.imshow(binary)
plt.show()

In [ ]:
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = img[400:-400,1300:-1300]
    _, binary = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)

    cnts, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    return binary, cnts


images_paths=[
            #   'data/images/0005.JPG',
              'data/images/0500.JPG',
              'data/images/1000.JPG',
              'data/images/1500.JPG',
              'data/images/2000.JPG',
              'data/images/2500.JPG',
              'data/images/3000.JPG',
              'data/images/3500.JPG',
              'data/images/4000.JPG',
              'data/images/4321.JPG',
              ]

images_names=[os.path.basename(image_path) for image_path in images_paths]



# Contour length

In [ ]:
%%time

# Read precomputed contour metrics from Parquet
in_path = Path("poliamid_group_250.parquet")

table = pq.read_table(in_path)
data = table.to_pydict()



In [ ]:


n = len(data["group_label"])
groups = [{k: data[k][i] for k in data.keys()} for i in range(n)]
group_labels = [g["group_label"] for g in groups]

# Which contour metric to plot
types = ['len_nodes', 'len_pixels', 'area']
type = types[2]

col_map = {
    "len_nodes": "len_nodes_list",
    "len_pixels": "len_pixels_list",
    "area": "area_list",
}

for step in tqdm([1, 2, 3, 4, 5]):

    fig, axes = plt.subplots(2, 4, figsize=(60, 20))

    # Colors per group
    colors = plt.cm.tab20(np.linspace(0, 1, max(2, min(20, len(groups)))))

    n_list = []
    p_list = []
    lambda_list = []
    mu_list = []
    sigma_list = []
    a_list = []

    for idx, g in enumerate(groups):
        len_list = g.get(col_map[type], [])
        if len_list is None:
            len_list = []
        else:
            len_list = list(len_list)

        if not len_list:
            continue

        x_orig, y_orig = cstats.stats_preprocess(len_list, step)

        # Calculate mode (x value with maximum y)
        mode_idx = np.argmax(y_orig)
        mode = x_orig[mode_idx]

        color = colors[idx % len(colors)]

        # Fit binomial distribution
        (x_fit, y_fit), n_fit, p_fit, amp = capprox.binomial_approx(
            x_orig, y_orig, n=25, p=0.2, x_lim=[0, 25], N=100
        )
        axes[0, 0].plot(x_fit, y_fit, ':', linewidth=2, color=color)

        # Fit Poisson distribution
        (x_fit, y_fit), lam, amp_poisson = capprox.poisson_approx(
            x_orig, y_orig, x_lim=[-5, 25], N=100
        )
        axes[0, 1].plot(x_fit, y_fit, ':', linewidth=2, color=color)

        # Fit Gaussian distribution
        (x_fit, y_fit), mu, sigma, amp_gauss = capprox.gauss_approx(
            x_orig, y_orig, mu=3, sigma=3, amp=1, x_lim=[0, 25], N=100
        )
        axes[0, 2].plot(x_fit, y_fit, '-.', linewidth=2, color=color)

        # Fit Exponential distribution
        (x_fit, y_fit), a, amp_exp = capprox.exponential_approx(
            x_orig, y_orig, a=5, amp=1, x_lim=[0, 25], N=100
        )
        axes[0, 3].plot(x_fit, y_fit, '--', linewidth=2, color=color)

        axes[0, 0].set_title(f'binomial fit, step={step}', fontsize=15)
        axes[0, 1].set_title(f'poisson fit, step={step}', fontsize=15)
        axes[0, 2].set_title(f'gaussian fit, step={step}', fontsize=15)
        axes[0, 3].set_title(f'exponential fit, step={step}', fontsize=15)

        n_list.append(n_fit)
        p_list.append(p_fit)
        lambda_list.append(lam)
        mu_list.append(mu)
        sigma_list.append(sigma)
        a_list.append(a)

        group_label = g["group_label"]

        # Plot original data with the same color (include mode in legend)
        axes[0, 0].plot(x_orig, y_orig, '-o', color=color, label=f'{group_label}, m={mode:.0f}, n={n_fit:.0f}, p={p_fit:.3f}')
        axes[0, 1].plot(x_orig, y_orig, '-o', color=color, label=f'{group_label}, m={mode:.0f}, λ={lam:.2f}')
        axes[0, 2].plot(x_orig, y_orig, '-o', color=color, label=f'{group_label}, m={mode:.0f}, μ={mu:.2f}, σ={sigma:.2f}')
        axes[0, 3].plot(x_orig, y_orig, '-o', color=color, label=f'{group_label}, m={mode:.0f}, a={a:.2f}')

    axes[1, 0].set_title(f'binomial fit, step={step}', fontsize=15)
    axes[1, 1].set_title(f'poisson fit, step={step}', fontsize=15)
    axes[1, 2].set_title(f'gaussian fit, step={step}', fontsize=15)
    axes[1, 3].set_title(f'exponential fit, step={step}', fontsize=15)

    # Use dual y-axis for n and p (n on left, p on right)
    ax1 = axes[1, 0]
    ax1.plot(range(len(n_list)), n_list, '-o', label='n', color='blue')
    ax1.set_ylabel('n', color='blue', fontsize=15)
    ax1.tick_params(axis='y', labelcolor='blue', labelsize=15)

    ax2 = ax1.twinx()
    ax2.plot(range(len(p_list)), p_list, '-o', label='p', color='red')
    ax2.set_ylabel('p', color='red', fontsize=15)
    ax2.tick_params(axis='y', labelcolor='red', labelsize=15)
    axes[1, 0].twinx().set_yscale('log')

    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right', fontsize=15)

    axes[1, 1].plot(range(len(lambda_list)), lambda_list, '-o', label='λ')
    axes[1, 2].plot(range(len(mu_list)), mu_list, '-o', label='mu')
    axes[1, 2].plot(range(len(sigma_list)), sigma_list, '-o', label='sigma')
    axes[1, 3].plot(range(len(a_list)), a_list, '-o', label='a')

    for ax in [axes[1, 0], axes[1, 1], axes[1, 2], axes[1, 3]]:
        ax.set_xticks(range(len(groups)))
        ax.set_xticklabels(group_labels, rotation=45, fontsize=15, ha='right')

    axes[0, 0].legend(loc='upper right', fontsize=12)
    axes[0, 1].legend(loc='upper right', fontsize=12)
    axes[0, 2].legend(loc='upper right', fontsize=12)
    axes[0, 3].legend(loc='upper right', fontsize=12)
    axes[1, 1].legend(loc='upper right', fontsize=15)
    axes[1, 2].legend(loc='upper right', fontsize=15)
    axes[1, 3].legend(loc='upper right', fontsize=15)

    axes[0, 0].set_xlim(0, 25)
    axes[0, 1].set_xlim(0, 25)
    axes[0, 2].set_xlim(0, 25)
    axes[0, 3].set_xlim(0, 25)

    if type == 'len_nodes':
        axes[0, 0].set_xlabel('len of contour, nodes', fontsize=15)
        axes[0, 1].set_xlabel('len of contour, nodes', fontsize=15)
        axes[0, 2].set_xlabel('len of contour, nodes', fontsize=15)
        axes[0, 3].set_xlabel('len of contour, nodes', fontsize=15)
    elif type == 'len_pixels':
        axes[0, 0].set_xlabel('len of contour, pixels', fontsize=15)
        axes[0, 1].set_xlabel('len of contour, pixels', fontsize=15)
        axes[0, 2].set_xlabel('len of contour, pixels', fontsize=15)
        axes[0, 3].set_xlabel('len of contour, pixels', fontsize=15)
    elif type == 'area':
        axes[0, 0].set_xlabel('area of contour, pixels', fontsize=15)
        axes[0, 1].set_xlabel('area of contour, pixels', fontsize=15)
        axes[0, 2].set_xlabel('area of contour, pixels', fontsize=15)
        axes[0, 3].set_xlabel('area of contour, pixels', fontsize=15)

    axes[0, 0].set_ylabel('p(x)', fontsize=15)
    axes[0, 1].set_ylabel('p(x)', fontsize=15)
    axes[0, 2].set_ylabel('p(x)', fontsize=15)
    axes[0, 3].set_ylabel('p(x)', fontsize=15)

    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

    if type == 'len_nodes':
        plt.savefig(f'countour_lens_nodes_step={step}.jpg', bbox_inches='tight')
    elif type == 'len_pixels':
        plt.savefig(f'countour_lens_pixels_step={step}.jpg', bbox_inches='tight')
    elif type == 'area':
        plt.savefig(f'countour_areas_step={step}.jpg', bbox_inches='tight')

    plt.show()
